# 应用场景

K-Means是一种非监督学习,解决的是聚类问题.K代表的是K类,Means代表的是中心.换句话说K-Means的本质是确认K类的中心点,找到中心点也就完了聚类.

# 原理概念

## K-Means的工作原理

- 选取K个点作为初始的类中心点,这些点一般都是从数据集中随记抽取的
- 将每个点分配到最近的类中心点,这样就形成了K个类,然后重新计算每个类的中心点
- 重复第二部,直到类不发生变化,或者你也可以设置最大迭代次数,这样即使类中心点发生变化,但是只要达到最大迭代次数就会结束.

## K-Means和KNN两者比较

- 两个算法解决数据挖掘的两类问题.K-Means是聚类算法,KNN是分类算法
- 两个算法分别是两种不同的学习方式.K-Means是非监督学习,也就是不需要事先给出分类标签,而KNN是有监督学习,需要我们给出训练数据的分类标识
- K值的含义不同.K-Means中K值代表K类;KNN中的K值代表K个最接近的邻居

# 工具介绍

sklearn是Python的机器学习工具库,如果从功能来划分,skleran可以实现分类,聚类,回归,降维,模型选择和预处理等功能.

> from sklearn.cluster import KMeans

sklearn.cluster提供了9中聚类方法,比如Mean-shift,DBSCAN,Spectral clustering(谱聚类).

>KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm='auto')


K-Means主要参数介绍如下:
- n_clusters:即K值,一般需要多试一些K值来 保证更好的聚类效果.通过随机设置一些K值,然后选择聚类效果最好的最为最终的K值;
- max_iter:最大迭代次数,如果聚类很难收敛的话,设置最大迭代次数可以让我们及时得到反馈结果,否则程序运行时间会非常长;
- n_init:初始化中心点的运算次数,默认是10.程序是否能快速收敛和中心点的选择关系非常大,所以在中心点上多花些时间,来争取整体时间上的快速收敛还是非常值得的.由于每一次中心点都是随机生成的,这样得到的结果有好有坏,非常不确定.所以通过设置n_inti次数,取其中最好的作为初始的中心点.如果K值比较大的时候,可适当增大n_init的值
- init:即初始值选择的方式,默认是采用优化过的k-means++方式,也可以自己指定中心点,或者采用random完全随机的方式.自己设置中心点一般是对于个性化数据进行设置,其他时候很少采用.random的方式则是完全随机,一般推荐采用优化过的k-means++方式;
- algorithm:k-means的实现算法,有'auto''full''elkan'三种.一般来说建议直接用默认的'auto'.如果选择'full'采用的是传统的K-Means算法;'auto'会根据数据的特点自动选择'full'或者'elkan.

# 项目流程

## 问题描述

对图像进行分割

图像分割就是利用图像自身的信息，比如颜色、纹理、形状等特征进行划分，将图像分割成不同的区域，划分出来的每个区域就相当于是对图像中的像素进行了聚类。

## 关键流程

因为处理的是图像信息,除了要获取图像数据以外,还需要获取图像的尺寸和通道数,然后基于图像中每个通道的数值进行数据规范化.这里通过定义一个load_data函数的方法来实现对图像进行加载和数据规范化.

In [1]:
# -*- coding: utf-8 -*-
# 使用K-means对图像进行聚类，显示分割标识的可视化
import numpy as np
import PIL.Image as image
from sklearn.cluster import KMeans
from sklearn import preprocessing
from skimage import color

>skimage是图像处理工具包,使用pip install scikit-image安装

In [2]:
# 加载图像，并对数据进行规范化
def load_data(filePath):
    # 读取文件
    f = open(filePath,'rb')
    data = []
    # 得到图像的像素值
    img = image.open(f)
    # 得到图像尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 得到点 (x,y) 的三个通道值
            c1, c2, c3 = img.getpixel((x, y))
            data.append([c1, c2, c3])
    f.close()
    # 采用 Min-Max 规范化
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    return np.mat(data), width, height

为了快速聚类的收敛,我们采用Min-Max规范化对数据进行规范化,自定义load_data函数返回的结果包括针对(R,G,B)三个通道规范化的数据以及图像的尺寸信息.

In [3]:
# 加载图像，得到规范化的结果 img，以及图像尺寸
img, width, height = load_data('./img/K-Means之美女与野兽.jpg')

In [4]:
# 用K-Means对图像进行2聚类
kmeans = KMeans(n_clusters=2)
kmeans.fit(img)
label = kmeans.predict(img)
# 将图像聚类结果,转化成图像尺寸的矩阵
label = label.reshape([width, height])
# 创建一个新图像pic_mark,用来保存图像聚类的结果,并设置不同的灰度值
pic_mark = image.new('L', (width, height))
for x in range(width):
    for y in range(height):
        # 根据类别设置图像灰度,类别0灰度值为255,类别1灰度值为127
        pic_mark.putpixel((x, y), int(256 / (label[x][y] + 1)) - 1)
pic_mark.save('美女与野兽_mark.jpg', 'JPEG')

![](./美女与野兽_mark.jpg)

使用fit和predict这两个函数来做数据的训练拟合和预测,因为传入的函数一样,我们可以同时进行fit和predict操作,然后使用fit_predict(data)得到聚类的结果(lable).label实际上是一个一维向量,需要将它转化成图像尺寸的矩阵.label的聚类结果是从0开始统计的,当聚类为2的时候,聚类的标识label=0或者1.

需要注意的是直接看0和1是看不出来的,所以要想对图像聚类的结果进行可视化,需要将0和1转化成灰度值.  
灰度值一般在0-255的范围内,可以将label=0设定为灰度值255,label=1设定为灰度值127,具体方法是用int(256/(lable[x][y]+1))-1 .这里可以看出,可视化的时候是通过设置图像的灰度值记性显示.经过转化以后原来图像的灰度值是0-255之间,现在只有2种颜色,即灰度255和127.

如果想要分割成16个部分,又该如何对不同分类设置不同的颜色值呢?  
这里需要用道skimage工具包,它是图像处理工具包,使用pip install scikit-image进行安装

In [5]:
# 用K-Means对图像进行16聚类
kmeans =KMeans(n_clusters=16)
kmeans.fit(img)
label = kmeans.predict(img)
# 将图像聚类结果，转化成图像尺寸的矩阵
label = label.reshape([width, height])
# 将聚类标识矩阵转化为不同颜色的矩阵
label_color = (color.label2rgb(label)*255).astype(np.uint8)
label_color = label_color.transpose(1,0,2)
images = image.fromarray(label_color)
images.save('美女与野兽_mark_color.jpg')

![](./美女与野兽_mark_color.jpg)

label2rgb函数将label分类标识转化为颜色数值,之所有乘以255是因为颜色值范围是[0,255],再转化为np.unit8类型.unit8类型代表无符号整数,范围是0-255之间.   
将得到的颜色矩阵输出后发现图像是点到的,需要设置三维矩阵转置,让第一维和第二维互换,使用transpose(1,0,2),相当于将原来(0,1,2)顺序转化为(1,0,2)顺序.  
最后使用fromarray函数,通过矩阵来生成图片并保存.

### 流程总结

不难发现K-Means聚类有个缺陷:K值需要事先指定.通常的做法是多设置几个K值,然后选择聚类结果最好的那个值.
以图像分割为例,可以看到K-Means计算你的过程在sklearn就是几行代码,大部分工作还是在预处理和后处理上.预处理是将图像进行加载,数据规范化;后处理是对聚类后的结果进行变换.
另外还学习到在Python中如何对图像进行读写.PIL工具包(Python Imaging Libary),即Python图像处理标准库.
注意不同尺寸的图像,K-Means运行的时间是不同的;如果图像尺寸比较大,可以事先进行压缩,长宽在200像素内运行速度会比较快;如果超过了1000像素,速度会很慢.

In [6]:
# import PIL.Image as image
# # 得到图像的像素值
# img = image.open(f)
# # 得到图像尺寸
# width, height = img.size